In [1]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter
import random
from python_speech_features import mfcc
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
from sklearn.mixture import GaussianMixture, _gaussian_mixture
reload(vad)

<module 'misc.vad' from 'C:\\Users\\David\\OneDrive\\Desktop\\pythonProjects\\ubm\\misc\\vad.py'>

In [12]:
# [1] separate data into train, enroll and verify

datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\speech_commands_v0.01"
main_ads = myads.AudioDatastore()
main_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)

In [13]:
ads = myads.subset(main_ads, label='stop')

speakers = []
for file in ads.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

ads.set(labels=speakers)

In [14]:
num_speakers_to_enroll = 10
label_count = Counter(ads.labels)
for_enroll_and_test_set = []
for key, cnts in list(label_count.items()):
    if cnts >= 3:
        for_enroll_and_test_set.append(key)
for_enroll = random.sample(for_enroll_and_test_set, num_speakers_to_enroll)
ads_enroll_and_validate = myads.subset(ads, label=for_enroll)
ads_enroll, _ = myads.split(ads_enroll_and_validate, 2)

ads_test = myads.subset(ads, for_enroll_and_test_set)
ads_test = myads.filter(ads_test, ads_enroll.files) # by file

ads_train_ubm = myads.filter(ads, ads_test.files)
ads_train_ubm = myads.filter(ads_train_ubm, ads_enroll.files)

In [15]:
# [2] feature extraction
# 	• Normalize the audio
# 	• Use detectSpeech to remove nonspeech regions from the audio
# 	• Extract features from the audio
# 	• Normalize the features
#   * Apply cepstral mean normalization

def helper_feature_extraction(audio_file, norm_factors = None):
    # read in file
    (rate,sig) = wav.read(audio_file)

    # normalise
    sig = sig / max(sig)

    # detect speech => convert this into one function
    v = vad.VoiceActivityDetector(rate, sig, 0.5)
    detected_speach = v.detect_speech()
    idx = v.convert_windows_to_readible_labels(detected_speach)
    if idx:
        sig = sig[idx[0]:idx[1]]
        mfcc_feat = mfcc(sig,rate)

        # feature normalisation and Cepstral mean subtraction (for channel noise)
        if norm_factors:
            mfcc_feat = (mfcc_feat - norm_factors.means) / norm_factors.std
            mfcc_feat = mfcc_feat - np.mean(mfcc_feat)

        return mfcc_feat
    else:
        return []


In [16]:
from sklearn.mixture import GaussianMixture

# wrapper class for sklearn's gmm

class MyGmm:
    def __init__(self, num_comp):
        self.gmm = GaussianMixture(n_components=num_comp, covariance_type='diag')
    def get_gmm(self):
        return self.gmm
    def set_gmm(self, some_gmm):
        self.gmm = some_gmm
    def set_gmm_params(self, w, m, c):
        # quick check
        num_comp = self.gmm.n_components
        if w.shape[0]!=num_comp or m.shape[0]!=num_comp or c.shape[0]!=num_comp:
            print('somethings wrong')
        else:
            self.gmm.weights_ = w
            self.gmm.means_ = m
            self.gmm.covariances_ = c
            self.gmm.precisions_cholesky_ = _gaussian_mixture._compute_precision_cholesky(c, 'diag')

In [17]:
import pickle

try:
    all_features = pickle.load(open('all_features.pickle','rb'))
except:
    all_features = []
    for file in ads.files:
        feature = helper_feature_extraction(file)
        all_features.append(feature)
    pickle.dump(all_features, open('all_features.pickle','wb'))


In [18]:
# collect normalization factors, why and where is this used ?

import numpy as np
# means = np.mean(all_features.fl, axis=0)
means = []
std = []
for feature in all_features:
    if len(feature) > 0:
        means.append(np.mean(feature, axis=0))
        std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.std(std, axis=0)

class NormFactor:
  def __init__(self, m, s):
    self.means = m
    self.std = s

normFactors = NormFactor(means, std)

print(normFactors.means)

[ -2.90282935  -0.33093011 -20.11337245 -14.39478988 -20.00294044
   1.69950359   2.69763417  -0.9433615   -8.80432994   2.05696167
  -4.85549398  -3.99526123  -9.69053713]


In [19]:
# normalise features, need to re-look at this ? how does cepstral mean normalization work ?

normalised_features = []
for i in range(len(all_features)):
    if len(all_features[i]) > 0:
        normalised = (all_features[i] - means) / std
        normalised = normalised - np.mean(normalised)
        normalised_features.append(normalised)
    else:
        normalised_features.append(np.array([]))


In [20]:
# [3] gmm ubm EM

# 3.a collect features
maxFeatures = len(ads_train_ubm.files) # len(
train_features = []
for f in range(maxFeatures):
    # extract a feature
    train_features.append(helper_feature_extraction(ads_train_ubm.files[f], normFactors))

In [21]:
# 3.b create UBM using sklearn

from sklearn.mixture import GaussianMixture, _base, _gaussian_mixture

numComponents = 32
alpha = np.ones(numComponents) / numComponents
numFeatures = normalised_features[0].shape[1]
mu = np.random.random((numComponents,numFeatures))
sigma = np.random.random((numComponents,numFeatures))

# TODO function to init GMM
ubm = GaussianMixture(n_components=numComponents, covariance_type='diag', weights_init=alpha, means_init=mu, precisions_init=sigma)

ubm.means_ = mu
ubm.covariances_ = sigma
ubm.weights_ = alpha
ubm.precisions_cholesky_ = _gaussian_mixture._compute_precision_cholesky(sigma, 'diag')

In [22]:
# 3.c em algorithm #todo something not right here double check
from scipy.special import logsumexp

maxIter = 100
targetLogLikelihood = 0
tol = 1e-3
pastL = -np.inf

for i in range(maxIter):

    # E step
    N = np.zeros((1, numComponents))
    F = np.zeros((numFeatures, numComponents))
    S = np.zeros((numFeatures, numComponents))
    L = 0
    eps = np.finfo(np.float64).eps

    for feature in train_features:
        # check for sub par samples
        if len(feature) > 0:
            logLikelihood = ubm._estimate_weighted_log_prob(feature)
            logLikelihoodSum = logsumexp(logLikelihood, axis=1)
            gamma = np.exp((logLikelihood.T - logLikelihoodSum)).T

            # Compute Baum-Welch stats
            n = np.sum(gamma,axis=0)
            f = np.dot(feature.T, gamma)
            s = np.dot(feature.T * feature.T, gamma)

            N = N + n
            F = F + f
            S = S + s

            L = L + np.sum(logLikelihoodSum)

    # M step
    L = L / len(train_features)
    if i%10 == 0:
        print('iter: ' + str(i) + ', log likelihood ' + str(L))
    if L > targetLogLikelihood or np.abs(pastL - L) < tol:
        print('converged on iter: ' + str(i))
        ubm.converged_ = True
        break
    else:
        pastL = L

    # TODO create function out of this
    N = np.maximum(N, eps)
    weights = np.maximum(N / np.sum(N), eps)
    newWeights = weights / np.sum(weights)  # why again, because of eps ...
    newMeans = F / N
    newCovariances = np.maximum((S / N) - np.square(ubm.means_).T, eps)
    ubm.weights_ = np.squeeze(newWeights.T)
    ubm.means_ = newMeans.T
    ubm.covariances_ = newCovariances.T


iter: 0, log likelihood -6664.798061316596
iter: 10, log likelihood -5278.89807991309
iter: 20, log likelihood -5226.25744729094
iter: 30, log likelihood -5205.135872586785
iter: 40, log likelihood -5193.571263263209
iter: 50, log likelihood -5185.511126132337
iter: 60, log likelihood -5181.354714833754
iter: 70, log likelihood -5177.664894153763
iter: 80, log likelihood -5173.590992279247
iter: 90, log likelihood -5170.6458312371715


In [23]:
# sklearn does better - probably because of k means init

ubm_test = GaussianMixture(n_components=32, covariance_type='diag')
train_features_flattened = np.array([item for sublist in train_features for item in sublist])
ubm_test.fit(train_features_flattened)
print('sklearn model' + str(ubm_test.score(train_features[0])))
print('my model' + str(ubm.score(train_features[0])))

sklearn model-37.22446257391194
my model-248.3206452951756


In [24]:
# save model

try:
    ubm = pickle.load(open('ubm.pickle','rb'))
except:
    print('train ubm not avaliable')
    pickle.dump(ubm, open('ubm.pickle','wb'))


In [25]:
# [4] map enroll
import time

t = time.time()
# do stuff
relevanceFactor = 16
speakers = np.unique(ads_enroll.labels)
numSpeakers = len(speakers)
gmmCellArray = []

for i in range(len(speakers)):
    adsTrainSubset = myads.subset(ads_enroll, speakers[i])
    N = np.zeros((1, numComponents))
    F = np.zeros((numFeatures, numComponents))
    S = np.zeros((numFeatures, numComponents))

    for file in adsTrainSubset.files:
        speaker_feature = helper_feature_extraction(file, normFactors)

        # BW stats, # TODO make this in to function
        logLikelihood = ubm._estimate_weighted_log_prob(speaker_feature)
        logLikelihoodSum = logsumexp(logLikelihood, axis=1)
        gamma = np.exp((logLikelihood.T - logLikelihoodSum)).T

        # Compute Baum-Welch stats
        n = np.sum(gamma,axis=0)
        f = np.dot(speaker_feature.T, gamma)
        s = np.dot(speaker_feature.T * speaker_feature.T, gamma)
        l = np.sum(logLikelihoodSum)

        N = N + n
        F = F + f
        S = S + s

    N = np.maximum(N, eps)
    weights = np.maximum(N / np.sum(N), eps)
    newWeights = weights / np.sum(weights)  # why again, because of eps ...
    newMeans = F / N
    newCovariances = np.maximum((S / N) - np.square(ubm.means_).T, eps)

    # create new speaker gmm
    gmm = GaussianMixture(n_components=numComponents, covariance_type='diag')
    gmm.weights_ = np.squeeze(newWeights.T)
    gmm.means_ = newMeans.T
    gmm.covariances_ = newCovariances.T
    gmm.precisions_cholesky_ = _gaussian_mixture._compute_precision_cholesky(newCovariances.T, 'diag')

    # calculate alpha
    alpha = N / (N + relevanceFactor)

    # adapt the means
    gmm.means_ = ((alpha*gmm.means_.T) + ((1-alpha)*ubm.means_.T)).T

    # Adapt the variances
    sigma = alpha*(S/N) + (1-alpha)*(ubm.covariances_ + np.square(ubm.means_)).T - np.square(gmm.means_).T
    gmm.covariances_ = np.maximum(sigma,eps).T

    # Adapt the weights
    weights = alpha*(N/np.sum(N)) + (1-alpha)*ubm.weights_.T
    gmm.weights_ = np.squeeze(weights/np.sum(weights))

    gmmCellArray.append(gmm)

elapsed = time.time() - t
print('speaker enrollment completed in: ', str(elapsed))

speaker enrollment completed in:  1.7823529243469238


In [27]:
enrolled_gmms = {}
for i in range(len(gmmCellArray)):
    enrolled_gmms[speakers[i]] = gmmCellArray[i]

In [32]:
# false rejection rate -
# The speaker false rejection rate (FRR) is the rate that a given speaker is incorrectly rejected. Use the known speaker set to determine the speaker false rejection rate for a set of thresholds.

llr = []

for i in range(len(speakers)):
    cur_speaker = speakers[i]
    local_gmm = enrolled_gmms[cur_speaker]
    adsTestSubset = myads.subset(ads_test, cur_speaker)
    llrPerSpeaker = np.zeros(len(adsTestSubset.files))

    for y in range(len(adsTestSubset.files)):
        file = adsTestSubset.files[y]
        speaker_feature = helper_feature_extraction(file, normFactors)

        logLikelihood = local_gmm._estimate_weighted_log_prob(speaker_feature)
        lspeaker = logsumexp(logLikelihood, axis=1)

        logLikelihood = ubm._estimate_weighted_log_prob(speaker_feature)
        lubm = logsumexp(logLikelihood, axis=1)

        llrPerSpeaker[y] = np.mean(lspeaker-lubm)

    llr.append(llrPerSpeaker)
